<a href="https://colab.research.google.com/github/April-LY/prublue/blob/master/new_edit_sinceri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
from scipy import stats

In [ ]:
class DATA():
  def __init__(self, time, num_time_points, totDATA, timeline, numGENES, genes,singleCELLdata):
    self.time = time
    self.num_time_points = num_time_points
    self.totDATA = totDATA
    self.timeline = timeline
    self.numGENES = numGENES
    self.genes = genes
    self.singleCELLdata = singleCELLdata
    ''' a error maybe the siglecelldata format'''

In [ ]:
def uploading():
  Whole_Data = pd.read_csv('/content/THP1_single_cell_data.csv')
  Whole_Data = pd.DataFrame(Whole_Data)

  #acquire gene name and the number of genes
  genes = Whole_Data.keys()
  genes = genes[0:-1]
  numGENES = len(genes)

  #delete the na column
  '''Q:different pesudotime the na value of the same cell
  will the condition be the same'''
  Whole_Data.dropna(axis = 1,how = "any",inplace = True)


  #totDATA delete the column of time--the last column
  Whole_Data = Whole_Data.values
  totData = Whole_Data[:,0:-1]

  #timeline extract all of the time point
  timeline = Whole_Data[:,-1]
  
  #Data.time euqals to different time type
  #number of different pesudotime
  time = list(set(timeline))
  time.sort()
  print("-----------")
  print(time)
  num_time_points = len(time)

  sortTOTdata = []
  sortTIMELINE = []
  
  for k in range(num_time_points):
    I = np.where(timeline == time[k])
    sortTOTdata.append(totData[I,:])
    sortTIMELINE.append(timeline[I])
  print(sortTOTdata)
  print(sortTIMELINE[0])

  totDATA=sortTOTdata
  timeline=sortTIMELINE

  '''lack the function of adding the default name'''
  singleCELLdata = sortTOTdata
  dt = DATA(time, num_time_points, totData, timeline, numGENES, genes, singleCELLdata)
  return dt

DATA = uploading() 

In [ ]:
from glmnet import glmnet
from cvglmnet import cvglmnet
from cvglmnetPlot import cvglmnetPlot

In [ ]:
def SINCERIETIES(DATA,distance = 1,method = 1,noDIAG = 0,SIGN = 1):

  ##Check input arguments
  if distance not in range(1,4):
    print("Choose distance metric with 1,2 and 3: 1 for Kolmogorov-Smirnov(KM), 2 for Cramer-von Mises(CM), 3 for Anderson-Darling(AD)")

  ##Initialization
  time = DATA.time
  single_cell_data = DATA.singleCELLdata
  numGENES = DATA.numGENES
  num_time_points = len(time)

  ##catch error

  #Distribution distance
  #Distance_matrix = np.arange(120).reshape(num_time_points-1,numGENES)
  Distance_matrix = np.zeros(shape = (num_time_points-1,numGENES))
  
  #print(Distance_matrix)
  totalDATA = single_cell_data[0]
  print(totalDATA) 
  #print(single_cell_data[1])

  '''cmtest2'''

  for ti in range(0,num_time_points-1):
    totalDATA = np.append(totalDATA,single_cell_data[ti+1])
    data_ti = single_cell_data[ti].T
    data_ti_plus1 = single_cell_data[ti+1].T
    
    for gi in range(0,numGENES):
      p1 = data_ti[gi,:]
      p2 = data_ti_plus1[gi,:]
      if distance == 1:
        p1 = p1.flatten()
        p2 = p2.flatten()
        #print(p2)
        test_stat = stats.ks_2samp(p1,p2)
        #print(test_stat.statistic)
        Distance_matrix[ti,gi] = test_stat.statistic
    
  ######################
  ##normalization
  #####################
  print(DATA.time[1:])
  print(DATA.time[0:-1])
  templist = list(map(lambda x: x[0]-x[1], zip(DATA.time[1:], DATA.time[0:-1])))
  templist
  dim1 = np.shape(Distance_matrix)[1]
  dim2 = np.shape(Distance_matrix)[0]
  flatten_temp_array = templist * dim1
  #print(x)
  #print(y)
  deltaT = np.reshape(flatten_temp_array,(dim1,dim2)).T
  Distance_matrix_normed = Distance_matrix/deltaT
  #print(Distance_matrix_normed)
  #print(deltaT)

  #############################
  #Generate Y and X_matrix for glmnet
  #method = 1 ridge
  #############################
  
  alphas = [0]
  print(len(alphas))
  Distance_matrix = Distance_matrix_normed
  X_matrix = Distance_matrix[0:num_time_points-2,]
  #print(X_matrix)

  ###################
  #LOOCV settings
  ####################
  nfold = np.shape(X_matrix)[0]
  foldid = range(1,nfold+1)
  keep = True
  print(numGENES)
  pred_lambda_min = np.zeros((numGENES,numGENES))
  print(pred_lambda_min)
  '''  
  lack elseif
  print(totalDATA) 
  a = range(0,num_time_points-1)
  print(a)
  '''  
  for gi in range(0,numGENES):
    cvlambda = []
    cvERROR = []
    beta = np.zeros((np.shape(X_matrix)[1],len(alphas)))

    for test in range(0,len(alphas)):
      Y_vector = Distance_matrix[1:num_time_points-1,gi]
      #new
      CV_results = cvglmnet(X_matrix, Y_vector,nfolds = nfold, foldid = foldid, keep = keep,grouped = False)
      #X_matrix,Y_vector,alpha=alphas[test],exclude=gi,nfolds = nfold, foldid = foldid,
          #keep = keep, lower.limits=0, upper.limits=Inf, grouped = FALSE
      cvglmnetPlot(CV_results)





In [ ]:
SINCERIETIES(DATA)